# Fine Tuning Model Open AI

Notebook ini didasarkan pada panduan terbaru yang diberikan di dokumentasi [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) dari Open AI.

Fine-tuning meningkatkan performa model dasar untuk aplikasi Anda dengan melatih ulang model menggunakan data tambahan dan konteks yang relevan untuk kasus penggunaan atau skenario tertentu. Perlu diketahui bahwa teknik rekayasa prompt seperti _few shot learning_ dan _retrieval augmented generation_ memungkinkan Anda memperkaya prompt default dengan data relevan untuk meningkatkan kualitas. Namun, pendekatan ini dibatasi oleh ukuran jendela token maksimum dari model dasar yang digunakan.

Dengan fine-tuning, kita secara efektif melatih ulang model itu sendiri dengan data yang dibutuhkan (sehingga kita bisa menggunakan lebih banyak contoh daripada yang bisa dimuat dalam jendela token maksimum) - dan menerapkan versi _custom_ dari model yang tidak lagi membutuhkan contoh saat proses inferensi. Ini tidak hanya meningkatkan efektivitas desain prompt kita (kita jadi lebih fleksibel dalam menggunakan jendela token untuk hal lain) tapi juga berpotensi menurunkan biaya (dengan mengurangi jumlah token yang perlu dikirim ke model saat inferensi).

Fine tuning terdiri dari 4 langkah:
1. Siapkan data pelatihan dan unggah.
1. Jalankan proses pelatihan untuk mendapatkan model yang sudah di-fine-tune.
1. Evaluasi model yang sudah di-fine-tune dan lakukan iterasi untuk meningkatkan kualitas.
1. Deploy model yang sudah di-fine-tune untuk inferensi jika sudah puas dengan hasilnya.

Perlu dicatat bahwa tidak semua model dasar mendukung fine-tuning - [cek dokumentasi OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) untuk informasi terbaru. Anda juga bisa melakukan fine-tune pada model yang sebelumnya sudah di-fine-tune. Dalam tutorial ini, kita akan menggunakan `gpt-35-turbo` sebagai model dasar yang akan di-fine-tune. 

---


### Langkah 1.1: Siapkan Dataset Anda

Mari kita buat chatbot yang membantu Anda memahami tabel periodik unsur dengan menjawab pertanyaan tentang suatu unsur dalam bentuk limerik. Dalam tutorial _sederhana_ ini, kita hanya akan membuat dataset untuk melatih model dengan beberapa contoh respons yang menunjukkan format data yang diharapkan. Dalam kasus penggunaan nyata, Anda perlu membuat dataset dengan jauh lebih banyak contoh. Anda juga bisa menggunakan dataset terbuka (untuk domain aplikasi Anda) jika tersedia, dan mengubah formatnya agar bisa digunakan untuk fine-tuning.

Karena kita fokus pada `gpt-35-turbo` dan menginginkan respons satu putaran (chat completion), kita bisa membuat contoh menggunakan [format yang disarankan ini](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) sesuai dengan kebutuhan chat completion OpenAI. Jika Anda mengharapkan konten percakapan multi-putaran, Anda akan menggunakan [format contoh multi-putaran](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) yang menyertakan parameter `weight` untuk menandai pesan mana yang digunakan (atau tidak) dalam proses fine-tuning.

Kita akan menggunakan format satu putaran yang lebih sederhana untuk tutorial ini. Datanya menggunakan [format jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) dengan 1 record per baris, masing-masing berupa objek berformat JSON. Cuplikan di bawah ini menunjukkan 2 record sebagai contoh - lihat [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) untuk set contoh lengkap (10 contoh) yang akan kita gunakan dalam tutorial fine-training ini. **Catatan:** Setiap record _harus_ didefinisikan dalam satu baris (tidak dipisah ke beberapa baris seperti pada file JSON yang diformat)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Dalam kasus penggunaan nyata, Anda akan membutuhkan set contoh yang jauh lebih besar untuk hasil yang baik - pertimbangannya adalah antara kualitas respons dan waktu/biaya untuk fine-tuning. Kita menggunakan set kecil agar proses fine-tuning bisa selesai dengan cepat untuk ilustrasi. Lihat [contoh OpenAI Cookbook ini](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) untuk tutorial fine-tuning yang lebih kompleks.


### Langkah 1.2 Unggah Dataset Anda

Unggah data menggunakan Files API [seperti dijelaskan di sini](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Perlu diperhatikan bahwa untuk menjalankan kode ini, Anda harus sudah melakukan langkah-langkah berikut:
 - Menginstal paket Python `openai` (pastikan Anda menggunakan versi >=0.28.0 untuk fitur terbaru)
 - Mengatur variabel lingkungan `OPENAI_API_KEY` ke API key OpenAI Anda
Untuk mempelajari lebih lanjut, lihat [Panduan Setup](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) yang disediakan untuk kursus ini.

Sekarang, jalankan kode untuk membuat file yang akan diunggah dari file JSONL lokal Anda.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Langkah 2.1: Buat pekerjaan Fine-tuning dengan SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Langkah 2.2: Periksa Status pekerjaan

Berikut beberapa hal yang bisa kamu lakukan dengan API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Menampilkan daftar n pekerjaan fine-tuning terakhir
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Mendapatkan detail dari pekerjaan fine-tuning tertentu
- `client.fine_tuning.jobs.cancel(<job_id>)` - Membatalkan pekerjaan fine-tuning
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Menampilkan hingga n event dari pekerjaan tersebut
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Langkah pertama dari proses ini adalah _memvalidasi file pelatihan_ untuk memastikan data sudah dalam format yang benar.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Langkah 2.3: Lacak event untuk memantau kemajuan


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Langkah 2.4: Lihat status di Dasbor OpenAI


Anda juga dapat melihat statusnya dengan mengunjungi situs web OpenAI dan menjelajahi bagian _Fine-tuning_ di platform tersebut. Di sana, Anda bisa melihat status pekerjaan yang sedang berjalan, serta melacak riwayat eksekusi pekerjaan sebelumnya. Pada tangkapan layar ini, Anda dapat melihat bahwa eksekusi sebelumnya gagal, dan percobaan kedua berhasil. Sebagai konteks, hal ini terjadi karena pada percobaan pertama digunakan file JSON dengan format record yang salah - setelah diperbaiki, percobaan kedua selesai dengan sukses dan model pun tersedia untuk digunakan.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.id.png)


Anda juga dapat melihat pesan status dan metrik dengan menggulir lebih jauh ke bawah di dasbor visual seperti yang ditunjukkan:

| Pesan | Metrik |
|:---|:---|
| ![Pesan](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.id.png) |  ![Metrik](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.id.png)|


### Langkah 3.1: Ambil ID & Uji Model yang Sudah Di-Fine-Tune di Kode


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Langkah 3.2: Memuat & Menguji Model Fine-Tuned di Playground

Sekarang kamu bisa menguji model yang sudah di-fine-tune dengan dua cara. Pertama, kamu bisa mengunjungi Playground dan menggunakan menu drop-down Models untuk memilih model fine-tuned milikmu dari daftar yang tersedia. Pilihan lainnya adalah menggunakan opsi "Playground" yang muncul di panel Fine-tuning (lihat screenshot di atas) yang akan membuka tampilan _perbandingan_ berikut, di mana versi model dasar dan model fine-tuned ditampilkan berdampingan untuk evaluasi cepat.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.id.png)

Cukup isi konteks sistem yang digunakan dalam data pelatihanmu dan masukkan pertanyaan uji coba. Kamu akan melihat kedua sisi diperbarui dengan konteks dan pertanyaan yang sama persis. Jalankan perbandingan dan kamu akan melihat perbedaan hasil keluaran di antara keduanya. _Perhatikan bagaimana model fine-tuned memberikan respons dalam format yang kamu berikan di contoh, sedangkan model dasar hanya mengikuti prompt sistem._

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.id.png)

Kamu juga akan melihat bahwa perbandingan ini menampilkan jumlah token untuk setiap model, serta waktu yang dibutuhkan untuk inferensi. **Contoh khusus ini memang sengaja dibuat sederhana untuk menunjukkan prosesnya, namun tidak benar-benar mencerminkan dataset atau skenario dunia nyata**. Kamu mungkin memperhatikan bahwa kedua sampel menunjukkan jumlah token yang sama (konteks sistem dan prompt pengguna identik) dengan model fine-tuned membutuhkan waktu lebih lama untuk inferensi (model kustom).

Dalam skenario nyata, kamu tidak akan menggunakan contoh sederhana seperti ini, melainkan melakukan fine-tuning dengan data asli (misalnya, katalog produk untuk layanan pelanggan) di mana kualitas respons akan jauh lebih terlihat. Dalam konteks _tersebut_, untuk mendapatkan kualitas respons yang setara dengan model dasar akan membutuhkan rekayasa prompt yang lebih khusus, yang akan meningkatkan penggunaan token dan kemungkinan waktu pemrosesan inferensi. _Untuk mencoba sendiri, lihat contoh fine-tuning di OpenAI Cookbook sebagai permulaan._



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berupaya untuk memberikan terjemahan yang akurat, harap diketahui bahwa terjemahan otomatis dapat mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang berwenang. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa terjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang timbul akibat penggunaan terjemahan ini.
